# Draft 3

Take only top 20 stocks and put full OHLCV data with the sentiment score to take advantage of more data points.


In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import yfinance as yf
import openai
from os import getenv
from dotenv import load_dotenv
import openai
import time
import seaborn as sns


# top 20 symbols

In [23]:
symbols_top20 = pd.read_csv('symbols/symbols_top_20.csv')

In [24]:
symbols_top20.loc[symbols_top20['Symbol'] == 'AAPL']

,Symbol,Dollar_Symbol,Name
0,AAPL,$AAPL,Apple Inc. Common Stock


In [21]:
symbols_top20.head(20)

,Symbol,Dollar_Symbol,Name
0,AAPL,$AAPL,Apple Inc. Common Stock
1,AMC,$AMC,AMC Entertainment Holdings Inc. Class A Common...
2,AMD,$AMD,Advanced Micro Devices Inc. Common Stock
3,CVNA,$CVNA,Carvana Co. Class A Common Stock
4,DISH,$DISH,DISH Network Corporation Class A Common Stock
5,HOOD,$HOOD,Robinhood Markets Inc. Class A Common Stock
6,MSFT,$MSFT,Microsoft Corporation Common Stock
7,NVDA,$NVDA,NVIDIA Corporation Common Stock
8,PYPL,$PYPL,PayPal Holdings Inc. Common Stock
9,RIVN,$RIVN,Rivian Automotive Inc. Class A Common Stock


# get the df

In [34]:
hot_df = pd.read_csv('text_df/hot_df.csv')
print(f"Hot length: {len(hot_df)}")
new_df = pd.read_csv('text_df/new_df.csv')
print(f"New length: {len(new_df)}")

Hot length: 3163
New length: 3327


In [26]:
new_df['text'][0]

'Wrinkle-brain Plays (Mathematically derived options plays)'

# get ticker

In [27]:
def symbol_matches(sentence, symbol_list):
    matches = []
    for symbol in symbol_list:
        pattern = r"\b" + re.escape(str(symbol)) + r"\b"
        if re.search(pattern, sentence):
            matches.append(symbol)
    return matches

def find_matches(hot_df, symbols):
    # List to store the results
    results = []

    # Iterate through each row in the DataFrame
    for index, row in hot_df.iterrows():
        date = row['date']
        sentence = str(row['text'])

        # Find matches for each sentence with "Symbol" and "Dollar_Symbol" columns
        matches_symbols = symbol_matches(sentence, symbols['Symbol'].tolist())
        matches_dollar_symbols = symbol_matches(sentence, symbols['Dollar_Symbol'].tolist())
        all_matches = matches_symbols + matches_dollar_symbols

        #if more than one match, revert to 'S&P'
        if len(all_matches) > 1:
            all_matches = ['S&P']
        # If all_matches is empty then input 'S&P'
        elif not all_matches:
            all_matches = ['S&P']



        ticker_string = ', '.join(all_matches)

        # Append the results to the list
        results.append((date, sentence, ticker_string))

    # Create a new DataFrame to store the results
    tickers = pd.DataFrame(results, columns=["date", "text", "ticker"])

    # Print the result DataFrame
    print(tickers)

    # Return the result DataFrame  
    return tickers


In [35]:
hot_ticker_top13 = find_matches(hot_df,symbols_top20)

            date                                               text ticker
0     2023-03-16       What Are Your Moves Tomorrow, March 16, 2023    S&P
1     2023-03-16  Most Anticipated Earnings Releases for the wee...    S&P
2     2023-03-16          Eggon says “GME go up” after earnings. ⬆️    S&P
3     2023-03-16  Goldman Sachs: 99% of borrowers have a mortgag...    S&P
4     2023-03-16                            Who would have thought?    S&P
...          ...                                                ...    ...
3158  2023-08-04                       When the IRS comes after you    S&P
3159  2023-08-04  How do people lose their entire account on opt...    S&P
3160  2023-08-04                  Better Option Strategies, Advice?    S&P
3161  2023-08-04                                 Whats up with $TGT    S&P
3162  2023-08-04  You do NOT have to buy the shares when exercis...    S&P

[3163 rows x 3 columns]


In [36]:
new_ticker_top13 = find_matches(new_df,symbols_top20)

            date                                               text ticker
0     2023-03-16  Wrinkle-brain Plays (Mathematically derived op...    S&P
1     2023-03-16     SONY Stock Forecast, Price & News (Sony Group)    S&P
2     2023-03-16                    Credit Suisse = Lehman Brothers    S&P
3     2023-03-16  Credit Suisse to borrow up to about $54 billio...    S&P
4     2023-03-16     "you met me at a very strange time in my life"    S&P
...          ...                                                ...    ...
3322  2023-08-04                         Why did you start trading?    S&P
3323  2023-08-04  MCRB - possibly groundbreaking microbiome trea...    S&P
3324  2023-08-04           Learned what early assignment is today 🥲    S&P
3325  2023-08-04     $250K YOLO on Allogene $ALLO High SI right now    S&P
3326  2023-08-04                  Better Option Strategies, Advice?    S&P

[3327 rows x 3 columns]


In [37]:
#write hot_ticker_top20 to csv file
hot_ticker_top13.to_csv('text_df/hot_ticker_top13.csv',index=False)
# write new_ticker_top20 
new_ticker_top13.to_csv('text_df/new_ticker_top13.csv',index=False)

In [38]:
hot_ticker_top13 = pd.read_csv('text_df/hot_ticker_top13.csv')
print(f"Hot length: {len(hot_ticker_top13)}")
new_ticker_top13 = pd.read_csv('text_df/new_ticker_top13.csv')
print(f"New length: {len(new_ticker_top13)}")

Hot length: 3163
New length: 3327


In [39]:
hot_ticker_top13['text'] = hot_ticker_top13['text'].str.lower()
new_ticker_top13['text'] = new_ticker_top13['text'].str.lower()
hot_ticker_top13.head()

,date,text,ticker
0,2023-03-16,"what are your moves tomorrow, march 16, 2023",S&P
1,2023-03-16,most anticipated earnings releases for the wee...,S&P
2,2023-03-16,eggon says “gme go up” after earnings. ⬆️,S&P
3,2023-03-16,goldman sachs: 99% of borrowers have a mortgag...,S&P
4,2023-03-16,who would have thought?,S&P


# get the tickers in a list


In [59]:
def price_tickers(ticker_df):
    # Count the occurrences of each ticker and sort them in descending order
    ticker_counts_df = ticker_df['ticker'].value_counts().sort_values(ascending=False).reset_index()

    # Reset the index and rename the columns for clarity
    ticker_counts_df.columns = ['Ticker', 'Count']
    # Filter the tickers that occur more than three times
    frequent_tickers_df = ticker_counts_df[ticker_counts_df['Count'] >= 10]
    print(frequent_tickers_df)
    # Convert the filtered tickers to a list
    frequent_tickers_list = frequent_tickers_df['Ticker'].tolist()

    # Return the list of frequent tickers
    return frequent_tickers_list

In [40]:
def ticker_list(df):
    ticker_list = list(set(df['ticker'].tolist()))
    print(ticker_list)
    return ticker_list

In [42]:
new_ticker_list = ticker_list(new_ticker_top13)


['CVNA', 'UBS', 'AMC', 'HOOD', 'DISH', 'AAPL', 'MSFT', 'AMD', 'TSLA', 'S&P', 'RTX', 'RIVN', 'NVDA', 'PYPL']


In [43]:
hot_ticker_list = ticker_list(hot_ticker_top13)

['CVNA', 'UBS', 'AMC', 'HOOD', 'DISH', 'AAPL', 'MSFT', 'AMD', 'TSLA', 'S&P', 'RTX', 'RIVN', 'NVDA', 'PYPL']


In [100]:
def download_stock_data(ticker):
    start_date = pd.to_datetime('2022-08-02')
    end_date = pd.to_datetime('2023-08-17')
    #download the stock
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

def download_ticker(ticker_list):
    for ticker in ticker_list:
        print(ticker)
        # Call the download_stock_data function to get adjusted closing prices
        stock_data = download_stock_data(ticker)
        #write the stock data to a csv file in the price_data folder
        stock_data.to_csv('price_data/top_13/'+ticker+'.csv')

In [101]:
download_ticker(hot_ticker_list)

UBS
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AMC
[*********************100%***********************]  1 of 1 completed
CVNA
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
HOOD
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
RTX
[*********************100%***********************]  1 of 1 completed
PYPL
[*********************100%***********************]  1 of 1 completed
DISH
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
S&P
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['S&P']: Exception('%ticker%: No timezone found, symbol may be delisted')



RIVN
[*********************100%***********************]  1 of 1 completed


In [102]:
download_ticker(new_ticker_list)

UBS
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AMC
[*********************100%***********************]  1 of 1 completed
CVNA
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
HOOD
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
RTX
[*********************100%***********************]  1 of 1 completed
PYPL
[*********************100%***********************]  1 of 1 completed
DISH
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
S&P
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['S&P']: Exception('%ticker%: No timezone found, symbol may be delisted')



RIVN
[*********************100%***********************]  1 of 1 completed


In [110]:
#test S&P 500 since the symbol in yf is unique
sp = download_stock_data("^GSPC")
sp.head()
#write it to a csv file
sp.to_csv('price_data/top_13/S&P.csv')

[*********************100%***********************]  1 of 1 completed


In [19]:
def read_price_data_top13(ticker_list):
    data_dict = {}
    for ticker in ticker_list:
        file_path = os.path.join('price_data','top_13' ,f'{ticker}.csv')
        if os.path.exists(file_path):
            data_dict[ticker] = pd.read_csv(file_path)
        else:
            print(f"File for {ticker} does not exist!")
    
    return data_dict

In [44]:
price_data_top13 = read_price_data_top13(hot_ticker_list)


In [45]:
price_data_top13['S&P'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-08-02,4104.209961,4140.470215,4079.810059,4091.189941,4091.189941,4727710000
1,2022-08-03,4107.959961,4167.660156,4107.959961,4155.169922,4155.169922,4351760000
2,2022-08-04,4154.850098,4161.290039,4135.419922,4151.939941,4151.939941,4283320000
3,2022-08-05,4115.870117,4151.580078,4107.310059,4145.189941,4145.189941,4085940000
4,2022-08-08,4155.930176,4186.620117,4128.970215,4140.060059,4140.060059,4221090000


# returns  calculated

In [46]:
def calculate_weekly_returns(price_data):
    for ticker, df in price_data.items():
        close_prices = df['Close']
        for days in range(1, 6):
            df[f'Returns_{days}'] = close_prices.pct_change(periods=days)
        price_data[ticker] = df
    return price_data

In [47]:
price_data_top_13 = calculate_weekly_returns(price_data_top13)
price_data_top_13['S&P'].head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Returns_1,Returns_2,Returns_3,Returns_4,Returns_5
0,2022-08-02,4104.209961,4140.470215,4079.810059,4091.189941,4091.189941,4727710000,NaN,NaN,NaN,NaN,NaN
1,2022-08-03,4107.959961,4167.660156,4107.959961,4155.169922,4155.169922,4351760000,0.015638,NaN,NaN,NaN,NaN
2,2022-08-04,4154.850098,4161.290039,4135.419922,4151.939941,4151.939941,4283320000,-0.000777,0.014849,NaN,NaN,NaN
3,2022-08-05,4115.870117,4151.580078,4107.310059,4145.189941,4145.189941,4085940000,-0.001626,-0.002402,0.013199,NaN,NaN
4,2022-08-08,4155.930176,4186.620117,4128.970215,4140.060059,4140.060059,4221090000,-0.001238,-0.002861,-0.003636,0.011945,NaN
5,2022-08-09,4133.109863,4137.299805,4112.089844,4122.470215,4122.470215,3913090000,-0.004249,-0.005481,-0.007098,-0.007870,0.007646
6,2022-08-10,4181.020020,4211.029785,4177.259766,4210.240234,4210.240234,4546010000,0.021291,0.016951,0.015693,0.014042,0.013253
7,2022-08-11,4227.399902,4257.910156,4201.410156,4207.270020,4207.270020,4630200000,-0.000705,0.020570,0.016234,0.014976,0.013326
8,2022-08-12,4225.020020,4280.470215,4219.779785,4280.149902,4280.149902,3788010000,0.017322,0.016605,0.038249,0.033838,0.032558
9,2022-08-15,4269.370117,4301.790039,4256.899902,4297.140137,4297.140137,3696830000,0.003970,0.021361,0.020640,0.042370,0.037941


# add to sentiment score

In [48]:
# read the sentiment files
hot_finbert = pd.read_csv('sentiment_scores/hot_finbert.csv')
hot_vader = pd.read_csv('sentiment_scores/hot_vader.csv')
hot_chatgpt = pd.read_csv('sentiment_scores/hot_chatgpt.csv')
hot_roberta = pd.read_csv('sentiment_scores/hot_roberta.csv')
new_finbert = pd.read_csv('sentiment_scores/new_finbert.csv')
new_vader = pd.read_csv('sentiment_scores/new_vader.csv')
new_chatgpt = pd.read_csv('sentiment_scores/new_chatgpt.csv')
new_roberta = pd.read_csv('sentiment_scores/new_roberta.csv')

In [51]:
def merge_dataframes(sentiment_df, price_data_dict):
    # List to store the merged rows
    merged_rows = []

    # Iterate through the rows in sentiment_df
    for index, row in sentiment_df.iterrows():
        date = row['date']
        text = row['text']
        ticker = row['ticker']
        label = row['label']
        score = row['score']

        # Check if the ticker exists in price_data_ret
        if ticker in price_data_dict:
            ticker_df = price_data_dict[ticker]
        else:
            # If ticker not found, use the 'S&P' DataFrame
            ticker_df = price_data_dict['S&P']
            ticker = 'S&P'
        
        matching_date = ticker_df[ticker_df['Date'] == date]

        

        # If a matching date is found, merge the entire row
        if not matching_date.empty:
            op = matching_date['Open'].iloc[0]
            hi = matching_date['High'].iloc[0]
            lo = matching_date['Low'].iloc[0]
            cl = matching_date['Close'].iloc[0]
            vol = matching_date['Volume'].iloc[0]
            ret_1 = matching_date['Returns_1'].iloc[0]
            ret_2 = matching_date['Returns_2'].iloc[0]
            ret_3 = matching_date['Returns_3'].iloc[0]
            ret_4 = matching_date['Returns_4'].iloc[0]
            ret_5 = matching_date['Returns_5'].iloc[0]

            merged_rows.append([date, text, ticker,label,score, op, hi, lo, cl, vol, ret_1, ret_2, ret_3, ret_4, ret_5])


    # Concatenate all the merged rows to create the merged DataFrame
    merged_df = pd.DataFrame(merged_rows, columns=['date','text','ticker','label', 'score', 'open','high','low','close','vol','ret_1','ret_2','ret_3','ret_4','ret_5'])

    return merged_df


In [52]:
hot_finbert_top_13 = merge_dataframes(hot_finbert,price_data_top_13)

In [53]:
hot_finbert_top_13.head()

,date,text,ticker,label,score,open,high,low,close,vol,ret_1,ret_2,ret_3,ret_4,ret_5
0,2023-03-16,"What Are Your Moves Tomorrow, March 16, 2023",S&P,neutral,0.901751,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
1,2023-03-16,Most Anticipated Earnings Releases for the wee...,S&P,neutral,0.846477,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
2,2023-03-16,Eggon says “GME go up” after earnings. ⬆️,S&P,neutral,0.465376,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
3,2023-03-16,Goldman Sachs: 99% of borrowers have a mortgag...,S&P,negative,0.776503,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
4,2023-03-16,Who would have thought?,S&P,neutral,0.783594,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709


In [54]:
new_finbert_top_13 = merge_dataframes(new_finbert,price_data_top_13)
new_finbert_top_13.head()

,date,text,ticker,label,score,open,high,low,close,vol,ret_1,ret_2,ret_3,ret_4,ret_5
0,2023-03-16,Wrinkle-brain Plays (Mathematically derived op...,S&P,neutral,0.931728,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
1,2023-03-16,"SONY Stock Forecast, Price & News (Sony Group)",S&P,neutral,0.917600,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
2,2023-03-16,Credit Suisse = Lehman Brothers,S&P,neutral,0.933757,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
3,2023-03-16,Credit Suisse to borrow up to about $54 billio...,S&P,neutral,0.818448,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709
4,2023-03-16,"""you met me at a very strange time in my life""",S&P,neutral,0.813300,3878.929932,3964.459961,3864.110107,3960.280029,5695790000,0.017562,0.010459,0.027108,0.025557,0.010709


In [55]:
hot_chagpt_t13 = merge_dataframes(hot_chatgpt,price_data_top_13)
new_chagpt_t13 = merge_dataframes(new_chatgpt,price_data_top_13)

hot_roberta_t13 = merge_dataframes(hot_roberta,price_data_top_13)
new_roberta_t13 = merge_dataframes(new_roberta,price_data_top_13)

In [56]:
hot_finbert_top_13.to_csv('sentiment_score_ret/top_13/hot_finbert_ret.csv',index=False)
new_finbert_top_13.to_csv('sentiment_score_ret/top_13/new_finbert_ret.csv',index=False)


hot_chagpt_t13.to_csv('sentiment_score_ret/top_13/hot_chatgpt_ret.csv',index=False)
new_chagpt_t13.to_csv('sentiment_score_ret/top_13/new_chatgpt_ret.csv',index=False)

hot_roberta_t13.to_csv('sentiment_score_ret/top_13/hot_roberta_ret.csv',index=False)
new_roberta_t13 .to_csv('sentiment_score_ret/top_13/new_roberta_ret.csv',index=False)

In [113]:
def merge_dataframes_vader(sentiment_df, price_data_dict):
    # List to store the merged rows
    merged_rows = []

    # Iterate through the rows in sentiment_df
    for index, row in sentiment_df.iterrows():
        date = row['date']
        text = row['text']
        ticker = row['ticker']
        compound = row['compound']

        # Check if the ticker exists in price_data_ret
        if ticker in price_data_dict:
            ticker_df = price_data_dict[ticker]
        else:
            # If ticker not found, use the 'S&P' DataFrame
            ticker_df = price_data_dict['S&P']
            ticker = 'S&P'
        
        matching_date = ticker_df[ticker_df['Date'] == date]

        

        # If a matching date is found, merge the entire row
        if not matching_date.empty:
            op = matching_date['Open'].iloc[0]
            hi = matching_date['High'].iloc[0]
            lo = matching_date['Low'].iloc[0]
            cl = matching_date['Close'].iloc[0]
            vol = matching_date['Volume'].iloc[0]
            ret_1 = matching_date['Returns_1'].iloc[0]
            ret_2 = matching_date['Returns_2'].iloc[0]
            ret_3 = matching_date['Returns_3'].iloc[0]
            ret_4 = matching_date['Returns_4'].iloc[0]
            ret_5 = matching_date['Returns_5'].iloc[0]

            merged_rows.append([date, text, ticker,compound, op, hi, lo, cl, vol, ret_1, ret_2, ret_3, ret_4, ret_5])


    # Concatenate all the merged rows to create the merged DataFrame
    merged_df = pd.DataFrame(merged_rows, columns=['date','text','ticker','compound', 'open','high','low','close','vol','ret_1','ret_2','ret_3','ret_4','ret_5'])

    return merged_df


In [115]:
hot_vader_top = merge_dataframes_vader(hot_vader,price_data_top_13)
new_vader_top = merge_dataframes_vader(new_vader,price_data_top_13)

In [117]:
#write hot_vader_top to csv file
hot_vader_top.to_csv('sentiment_score_ret/top_13/hot_vader_ret.csv',index=False)
#write new_vader_top to csv file
new_vader_top.to_csv('sentiment_score_ret/top_13/new_vader_ret.csv',index=False)

# price prediction

In [57]:
hot_finbert = pd.read_csv('sentiment_score_ret/top_13/hot_finbert_ret.csv')
hot_chatgpt = pd.read_csv('sentiment_score_ret/top_13/hot_chatgpt_ret.csv')
hot_roberta = pd.read_csv('sentiment_score_ret/top_13/hot_roberta_ret.csv')
new_finbert = pd.read_csv('sentiment_score_ret/top_13/new_finbert_ret.csv')
new_chatgpt = pd.read_csv('sentiment_score_ret/top_13/new_chatgpt_ret.csv')
new_roberta = pd.read_csv('sentiment_score_ret/top_13/new_roberta_ret.csv')

In [58]:
import tensorflow as tf
from keras.layers import Input, Embedding, LSTM, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [95]:
def plot_loss_and_accuracy(history):
    fig, axs = plt.subplots(1, 2, figsize=(12,5))
    
    # Plot training & validation accuracy values
    axs[0].plot(history.history['accuracy'])
    axs[0].plot(history.history['val_accuracy'])
    axs[0].set_title('Model accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set_title('Model loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['Train', 'Validation'], loc='upper left')
    
    plt.show()


In [111]:
def plot_week_eval(accuracy,precision,recall,loss):
  labels = ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5']
  fig, axes = plt.subplots(2, 2, figsize=(18, 12))
  # Accuracy
  axes[0, 0].plot(labels, accuracy, marker='o')
  axes[0, 0].set_title('Accuracy')
  axes[0, 0].grid(True)
  # Precision
  axes[0, 1].plot(labels, precision, marker='o')
  axes[0, 1].set_title('Precision')
  axes[0, 1].grid(True)
  #Recall
  axes[1, 0].plot(labels, recall, marker='o')
  axes[1, 0].set_title('Recall')
  axes[1, 0].grid(True)
  #Loss
  axes[1, 1].plot(labels, loss, marker='o')
  axes[1, 1].set_title('Loss')
  axes[1, 1].grid(True)
  
  plt.show()
